In [0]:
import os
import pandas as pd
import numpy as np
import xgboost as XGB
from xgboost.sklearn import XGBClassifier
from xgboost import XGBRanker
# from xgboostextension.scorer.metrics import ndcg
from sklearn.model_selection import GridSearchCV, GroupKFold
from xgboostextension.scorer import RankingScorer
from xgboostextension.scorer.metrics import ndcg
from google.colab import drive
from pprint import pprint
import gc

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
!pip uninstall xgboost
!pip install https://s3-us-west-2.amazonaws.com/xgboost-wheels/xgboost-1.0.2-py3-none-manylinux1_x86_64.whl

Uninstalling xgboost-1.0.2:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/xgboost-1.0.2.dist-info/*
    /usr/local/lib/python3.6/dist-packages/xgboost/*
Proceed (y/n)? n


In [0]:
!wget https://github.com/igorpejic/personalize_expedia_hotel_searches_2013/raw/5a9654f755399a83f4b40850e82c34648aab4b7a/run.py


--2020-05-20 16:58:59--  https://github.com/igorpejic/personalize_expedia_hotel_searches_2013/raw/5a9654f755399a83f4b40850e82c34648aab4b7a/run.py
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/igorpejic/personalize_expedia_hotel_searches_2013/5a9654f755399a83f4b40850e82c34648aab4b7a/run.py [following]
--2020-05-20 16:58:59--  https://raw.githubusercontent.com/igorpejic/personalize_expedia_hotel_searches_2013/5a9654f755399a83f4b40850e82c34648aab4b7a/run.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14884 (15K) [text/plain]
Saving to: ‘run.py’

run.py              100%[===================>]  

In [0]:

drive.mount('/content/gdrive')
# basedir = "H:\\dataset\\2nd-assignment-dmt-2020"
basedir = '/content/gdrive/My Drive/dm_dataset'
fileName = os.path.abspath(os.path.join(basedir, 'training_set_VU_DM.csv'))
fileName_test = os.path.abspath(os.path.join(basedir, 'test_set_VU_DM.csv'))


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:

expedia = pd.read_csv(fileName,low_memory=False)
test_df = pd.read_csv(fileName_test,low_memory=False)
print(expedia.shape)

In [0]:
columns_attrs = list(expedia.columns.values)
pprint(columns_attrs)

['srch_id',
 'date_time',
 'site_id',
 'visitor_location_country_id',
 'visitor_hist_starrating',
 'visitor_hist_adr_usd',
 'prop_country_id',
 'prop_id',
 'prop_starrating',
 'prop_review_score',
 'prop_brand_bool',
 'prop_location_score1',
 'prop_location_score2',
 'prop_log_historical_price',
 'position',
 'price_usd',
 'promotion_flag',
 'srch_destination_id',
 'srch_length_of_stay',
 'srch_booking_window',
 'srch_adults_count',
 'srch_children_count',
 'srch_room_count',
 'srch_saturday_night_bool',
 'srch_query_affinity_score',
 'orig_destination_distance',
 'random_bool',
 'comp1_rate',
 'comp1_inv',
 'comp1_rate_percent_diff',
 'comp2_rate',
 'comp2_inv',
 'comp2_rate_percent_diff',
 'comp3_rate',
 'comp3_inv',
 'comp3_rate_percent_diff',
 'comp4_rate',
 'comp4_inv',
 'comp4_rate_percent_diff',
 'comp5_rate',
 'comp5_inv',
 'comp5_rate_percent_diff',
 'comp6_rate',
 'comp6_inv',
 'comp6_rate_percent_diff',
 'comp7_rate',
 'comp7_inv',
 'comp7_rate_percent_diff',
 'comp8_rate',


In [0]:

cols = ['srch_length_of_stay', 'orig_destination_distance', 'prop_location_score1', 'srch_room_count',
        'prop_starrating', 'gross_bookings_usd', 'promotion_flag', 'prop_review_score', 'prop_location_score1', 'prop_location_score2']

labels_attrs = ['booking_bool', 'click_bool']
columns_attrs = list(expedia.columns.values)
usable_prefix = ['visitor', 'prop', 'promo', 'position', 'gross']
is_valid = lambda x: any([pref in x for pref in usable_prefix])
columns_attrs = [attr for attr in (columns_attrs) if is_valid(attr)]
# for a in labels_attrs:
    # columns_attrs.remove(a)
label_subset = expedia[labels_attrs]
exp_subset = expedia[cols]
corr = exp_subset.corr(label_subset)
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.rows.values)

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


ValueError: ignored

In [0]:
missing_data = []
for col in expedia.columns:
    #print(expedia[col].isnull().sum())
    missing_data.append([col, expedia[col].isnull().sum()])

In [0]:
#where the first column is the group ind, other columns are features
from run import load_data, preprocess_training_data
# load train data
train_data = load_data(fileName)
X, y = preprocess_training_data(train_data)

Started loading data from file /content/gdrive/My Drive/dm_dataset/training_set_VU_DM.csv
Finished loading data....
Preprocessing training data....
Dropping columns ['comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv', 'comp4_rate_percent_diff', 'comp6_rate', 'comp6_inv', 'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv', 'comp7_rate_percent_diff', 'gross_bookings_usd'] because they miss more than 0.9 of data.
Dropped columns ['comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv', 'comp4_rate_percent_diff', 'comp6_rate', 'comp6_inv', 'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv', 'comp7_rate_percent_diff', 'gross_bookings_usd']


In [0]:
# model

ranker = XGBRanker(objective='rank:pairwise', n_estimators=40, max_depth=3, learning_rate=0.1, subsample=0.5,
                   early_stopping_rounds=10, seed=5)
groups = X["srch_id"].value_counts(sort=False).sort_index()

In [0]:
yy = train_data['click_bool']+4*train_data['booking_bool']

In [0]:
X

NameError: ignored

In [0]:
grid_search = GridSearchCV(
    ranker,
    {
        'n_estimators': [5,40],
        'max_depth': [3,5,10]
    },
    cv=GroupKFold(5),
    scoring=RankingScorer(ndcg(5)),
    verbose=100,
    n_jobs=4
)
grid_search.fit(X, yy, groups=groups)
print("Cross validation results:")
print(grid_search.cv_results_)

In [0]:
# model
ranker = XGBRanker(objective='rank:ndcg', n_estimators=40, max_depth=3, learning_rate=0.1, subsample=0.5,
                   early_stopping_rounds=10, seed=5)


In [0]:
gc.collect()
grid_search = GridSearchCV(
    ranker,
    {
        'n_estimators': [40],
        'max_depth': [3]
    },
    cv=GroupKFold(5),
    scoring=RankingScorer(ndcg(5)),
    verbose=100,
    n_jobs=4
)
ranker.fit(X, yy, group=groups)
print("Cross validation results:")
# print(grid_search.cv_results_)

[17:12:15] WARNING: /workspace/src/gbm/gbtree.cc:138: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


In [0]:
!nvidia-smi